In [42]:
import pyomo.environ as pyo
import pyomo.opt as pyopt
from pyomo.core import Suffix

def read_cor_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    num_servers = 5
    num_clients = 25
    fixed_costs = [0] * num_servers
    revenues = {}
    capacity = 188

    for line in lines:
        parts = line.split()
        if len(parts) >= 3:
            if parts[0].startswith('x_') and parts[1] == 'obj':
                server = int(parts[0].split('_')[1]) - 1  # Adjust for 0-based indexing
                fixed_costs[server] = float(parts[2])
            elif parts[0].startswith('x_') and parts[1] == 'c':
                capacity = float(parts[2])
            elif parts[0].startswith('y_') and parts[1] == 'obj':
                client, server = map(int, parts[0].split('_')[1:3])
                client -= 1  # Adjust for 0-based indexing
                server -= 1  # Adjust for 0-based indexing
                revenue = -float(parts[2])  # Negate because we're maximizing
                revenues[(client, server)] = revenue

    print(f"Number of fixed costs: {len(fixed_costs)}")
    print(f"Number of revenue entries: {len(revenues)}")
    print(f"Capacity: {capacity}")
    print(f"Fixed costs: {fixed_costs}")
    print(f"First few revenue entries: {list(revenues.items())[:5]}")

    return num_servers, num_clients, fixed_costs, revenues, capacity

def read_sto_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    scenarios = {}
    current_scenario = None
    scenario_prob = None

    for line in lines:
        parts = line.split()
        if len(parts) >= 4 and parts[0] == 'SC':
            current_scenario = parts[1]
            scenario_prob = float(parts[3])
            scenarios[current_scenario] = {'prob': scenario_prob, 'clients': {}}
        elif len(parts) == 3 and parts[0] == 'RHS':
            print(str(len(parts)) + '_____________' + str(parts) + '@@@@@@@@@@@@@@@@@@@') #######################################
            client = int(parts[1][1:]) - 7  # c7 corresponds to client 0
            availability = int(parts[2])
            scenarios[current_scenario]['clients'][client] = availability

    # Print some debug information
    print(f"Number of scenarios: {len(scenarios)}")
    if scenarios:
        first_scenario = next(iter(scenarios.values()))
        print(f"Number of clients in first scenario: {len(first_scenario['clients'])}")
        print(f"Sample of first scenario data: {list(first_scenario['clients'].items())[:5]}")

    return scenarios


def create_model(num_servers, num_clients, fixed_costs, revenues, capacity, scenarios):
    model = pyo.ConcreteModel()

    # Sets
    model.I = pyo.RangeSet(0, num_clients-1)  # Clients
    model.J = pyo.RangeSet(0, num_servers-1)  # Servers
    model.S = pyo.Set(initialize=scenarios.keys())  # Scenarios

    # First-stage variables
    model.x = pyo.Var(model.J, domain=pyo.Binary)

    # Second-stage variables
    model.y = pyo.Var(model.I, model.J, model.S, domain=pyo.Binary)

    # Objective function
    def obj_rule(model):
        first_stage = sum(fixed_costs[j] * model.x[j] for j in model.J)
        second_stage = sum(scenarios[s]['prob'] * sum(revenues.get((i,j), 0) * model.y[i,j,s] 
                           for i in model.I for j in model.J) for s in model.S)
        return first_stage - second_stage  # Minimize costs - revenues

    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Constraints
    def capacity_rule(model, j, s):
        return sum(model.y[i,j,s] for i in model.I) <= capacity * model.x[j]

    model.capacity = pyo.Constraint(model.J, model.S, rule=capacity_rule)

    def assignment_rule(model, i, s):
        return sum(model.y[i,j,s] for j in model.J) <= scenarios[s]['clients'].get(i, 0)

    model.assignment = pyo.Constraint(model.I, model.S, rule=assignment_rule)

    # Maximum number of servers
    model.max_servers = pyo.Constraint(expr=sum(model.x[j] for j in model.J) <= 5)

    return model

def solve_model(model):
    solver = pyopt.SolverFactory('glpk')
    results = solver.solve(model, tee=True)

    if results.solver.status == pyopt.SolverStatus.ok and results.solver.termination_condition == pyopt.TerminationCondition.optimal:
        print("Optimal solution found")
        print("Opened servers:")
        for j in model.J:
            if pyo.value(model.x[j]) > 0.5:
                print(f"Server {j+1}")
        
        print("\nObjective value:", pyo.value(model.obj))
    else:
        print("Solver did not find an optimal solution")

def main():
    cor_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.cor'
    sto_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.sto'

    num_servers, num_clients, fixed_costs, revenues, capacity = read_cor_file(cor_file_path)
    scenarios = read_sto_file(sto_file_path)

    model = create_model(num_servers, num_clients, fixed_costs, revenues, capacity, scenarios)
    solve_model(model)

if __name__ == "__main__":
    main()
 # cor_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.cor'
    # sto_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.sto'


Number of fixed costs: 5
Number of revenue entries: 121
Capacity: 188
Fixed costs: [1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
First few revenue entries: [((0, 1), 22.0), ((0, 2), 18.0), ((0, 3), 14.0), ((0, 4), 22.0), ((1, 0), 15.0)]
3_____________['RHS', 'c7', '1']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c8', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c9', '1']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c10', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c11', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c12', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c13', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c14', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c15', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c16', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c17', '1']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c18', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c19', '1']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c20', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c21', 

# Experiment section

In [43]:
cor_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.cor'
sto_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.sto'

num_servers, num_clients, fixed_costs, revenues, capacity = read_cor_file(cor_file_path)
scenarios = read_sto_file(sto_file_path)

Number of fixed costs: 5
Number of revenue entries: 121
Capacity: 188
Fixed costs: [1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
First few revenue entries: [((0, 1), 22.0), ((0, 2), 18.0), ((0, 3), 14.0), ((0, 4), 22.0), ((1, 0), 15.0)]
3_____________['RHS', 'c7', '1']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c8', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c9', '1']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c10', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c11', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c12', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c13', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c14', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c15', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c16', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c17', '1']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c18', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c19', '1']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c20', '0']@@@@@@@@@@@@@@@@@@@
3_____________['RHS', 'c21', 

In [46]:
revenues

{(0, 1): 22.0,
 (0, 2): 18.0,
 (0, 3): 14.0,
 (0, 4): 22.0,
 (1, 0): 15.0,
 (1, 1): 11.0,
 (1, 2): 20.0,
 (1, 3): 8.0,
 (1, 4): 14.0,
 (2, 0): 4.0,
 (2, 1): 22.0,
 (2, 2): 10.0,
 (2, 4): 25.0,
 (3, 0): 14.0,
 (3, 1): 23.0,
 (3, 2): 23.0,
 (3, 3): 5.0,
 (3, 4): 22.0,
 (4, 0): 8.0,
 (4, 1): 23.0,
 (4, 2): 14.0,
 (4, 3): 5.0,
 (4, 4): 11.0,
 (5, 0): 18.0,
 (5, 1): 5.0,
 (5, 2): 2.0,
 (5, 3): 23.0,
 (5, 4): 6.0,
 (6, 0): 6.0,
 (6, 1): 8.0,
 (6, 2): 22.0,
 (6, 3): 3.0,
 (6, 4): 15.0,
 (7, 0): 14.0,
 (7, 1): 21.0,
 (7, 2): 6.0,
 (7, 3): 16.0,
 (7, 4): 14.0,
 (8, 0): 21.0,
 (8, 1): 6.0,
 (8, 2): 1.0,
 (8, 3): 8.0,
 (8, 4): 3.0,
 (9, 0): 16.0,
 (9, 1): 14.0,
 (9, 2): 13.0,
 (9, 3): 12.0,
 (9, 4): 22.0,
 (10, 0): 8.0,
 (10, 1): 20.0,
 (10, 2): 15.0,
 (10, 3): 15.0,
 (10, 4): 12.0,
 (11, 0): 11.0,
 (11, 1): 4.0,
 (11, 2): 9.0,
 (11, 3): 15.0,
 (11, 4): 11.0,
 (12, 0): 2.0,
 (12, 1): 19.0,
 (12, 2): 13.0,
 (12, 3): 2.0,
 (12, 4): 9.0,
 (13, 0): 15.0,
 (13, 1): 20.0,
 (13, 2): 17.0,
 (13, 4): 16.0